## 第11章 语言数据管理

已标注的语言数据的结构化集合在 NLP 的大部分领域都是至关重要的；但是，我们使用它们仍然面临着许多障碍。本章的目的是要回答下列问题： 
1. 我们如何设计一种新的语言资源，并确保它的覆盖面、平衡以及支持广泛用途的文档？
2. 现有数据对某些分析工具格式不兼容，我们如何才能将其转换成合适的格式？ 
3. 有什么好的方法来记录我们已经创建的资源的存在，让其他人可以很容易地找到它？

一路上，我们将研究当前语料库的设计、创建一个语料库的典型工作流程，及语料库的生命周期。与在其他章节中一样，会有语言数据管理实际实验的很多例子，包括在语言学现场教学课程、实验室的工作和网络爬取中收集的数据。


### 11.1 语料库结构：一个案例研究

`TIMIT`语料库是第一个广泛发布的已标注语音数据库，它有一个特别清晰的组织结构。
`TIMIT`由一个包括克萨斯仪器公司和麻省理工学院的财团开发，它也由此得名。它被设计用来为声学-语音知识的获取提供数据，并支持自动语音识别系统的开发和评估。

#### `TIMIT` 的结构

与布朗语料库显示文章风格和来源的平衡选集一样，`TIMIT`包括方言、说话者和材料的平衡选集。对 8个方言区中的每一种方言，具有一定年龄范围和教育背景的 50 个男性和女 性的说话者每人读 10个精心挑选的句子。

NLTK包括 `TIMIT` 语料库的一个样本。你可以使用常用的方式，也就是使用 `help(nltk.corpus.timit)`，访问它的文档。`nltk.corpus.timit.fileids()`可以看到语料库样本中 160 个录制的话语列表。每个文件名的内部结构如图11-1所示。
![11-1](./imgs/11-1.jpg)

每个项目都有音标，可以使用`phones()`方法访问。我们可以按习惯的方式访问相应的词标识符。两种访问方法都允许一个可选的参数 `offset=True`，其中包括音频文件的相应跨度的开始和结尾偏移。

In [1]:
import nltk
phonetic = nltk.corpus.timit.phones('dr1-fvmh0/sa1')
phonetic

['h#',
 'sh',
 'iy',
 'hv',
 'ae',
 'dcl',
 'y',
 'ix',
 'dcl',
 'd',
 'aa',
 'kcl',
 's',
 'ux',
 'tcl',
 'en',
 'gcl',
 'g',
 'r',
 'iy',
 's',
 'iy',
 'w',
 'aa',
 'sh',
 'epi',
 'w',
 'aa',
 'dx',
 'ax',
 'q',
 'ao',
 'l',
 'y',
 'ih',
 'ax',
 'h#']

In [2]:
# 除了这种文本数据，TIMIT 还包括一个词典，提供每一个词的可与一个特定的话语比较的规范的发音：
timitdict = nltk.corpus.timit.transcription_dict()
timitdict['greasy'] + timitdict['wash'] + timitdict['water']

['g', 'r', 'iy1', 's', 'iy', 'w', 'ao1', 'sh', 'w', 'ao1', 't', 'axr']

In [3]:
phonetic[17:30]

['g', 'r', 'iy', 's', 'iy', 'w', 'aa', 'sh', 'epi', 'w', 'aa', 'dx', 'ax']

In [4]:
nltk.corpus.timit.spkrinfo('dr1-fvmh0')

SpeakerInfo(id='VMH0', sex='F', dr='1', use='TRN', recdate='03/11/86', birthdate='01/08/60', ht='5\'05"', race='WHT', edu='BS', comments='BEST NEW ENGLAND ACCENT SO FAR')

#### 主要设计特点

`TIMIT`演示了语料库设计中的几个主要特点。首先，语料库包含**语音**和**字形**两个标注层。
一般情况下，文字或语音语料库可能在多个不同的语言学层次标注，包括**形态、句法和段落层次**。此外，即使在给定的层次仍然有不同的标注策略，甚至标注者之间也会有分歧，因此我们要表示多个版本。

`TIMIT` 的第二个特点是：它在多个维度的变化与方言地区和二元音覆盖范围之间取得平衡。人口学统计的加入带来了许多更独立的变量，这可能有助于解释数据 中的变化，便于以后出于在建立语料库时没有想到的目的使用语料库，例如社会语言学。

第三个特点是：将原始语言学事件作为录音来捕捉和作为标注来捕捉之间有明显的区分。两者 一致表示文本语料库正确，原始文本通常有被认为是不可改变的作品的外部来源。那个作品 的任何包含人的判断的转换——即使如分词一样简单——也是后来的修订版；因此以尽可能 接近原始的形式保留源材料是十分重要的。


第四个特点是语料库的层次结构。每个句子 4个文件，500 个说话者每人 10个句子，有 20,000个文件。这些被组织成一个树状结构。在顶层分成训练集和测试集，用于开发和评估统计模型。

最后，请注意虽然 `TIMIT` 是语音语料库，它的录音文本和相关数据只是文本，可以使用程序处理了，就像任何其他的文本语料库那样。因此，许多在这本书中所描述的计算方法都适用。此外，注意 `TIMIT` 语料库包含的所有数据类型分为**词汇**和**文字**两个基本类别，我们将在下面讨论。说话者人口学统计数据只不过是词汇数据类型的另一个实例。

#### 基本数据类型

不考虑它的复杂性，`TIMIT` 语料库只包含两种基本数据类型：词典和文本。。正如我们在第2 章中所看到的，大多数词典资源都可以使用记录结构表示，即一个关键字加一个或多个字段，如图11-3所示。词典资源可能是一个传统字典或比较词表，如下所示。它也可以是一个短语词典，其中的关键字是一个短语而不是一个词。词典还包括记录结构化的数据，我们可以通过对应主题的非关键字字段来查找条目。我们也可以构造特殊的表格（称为范例） 来进行对比和说明系统性的变化，图11-3显示了三个动词。TIMIT的说话者表也是一种词典资源。
![11-3](./imgs/11-3.jpg)



### 11.2 语料库生命周期

语料库并不是从天而降的，需要精心的准备和许多人长时期的输入。原始数据需要进行收集、清理、记录并以系统化的结构存储。标注可分为各种层次，一些需要语言的形态或句法的专门知识。要在这个阶段成功取决于建立一个高效的工作流程，包括适当的工具和格式转换器。质量控制程序可以将寻找标注中的不一致落实到位，确保尽最大可能在标注者之间达成一致。由于任务的规模和复杂性，大型语料库可能需要几年的准备，包括几十或上百人多年的努力。在本节中，我们简要地回顾语料库生命周期的各个阶段。


#### 语料库创建的三种方案

语料库的一种类型是设计在创作者的探索过程中逐步展现。这是典型的传统“领域语言
学”模式，即来自会话的材料在它被收集的时候就被分析，明天的想法往往基于今天的分析 中产生的问题。在随后几年的研究中产生的语料不断被使用，并可能用作不确定的档案资源。 计算机化明显有利于这种类型的工作，以广受欢迎的程序 Shoebox为例，它作为 Toolbox 重 新发布，现在已有超过二十年的历史（见 2.4 节）。其他的软件工具，甚至是简单的文字处 理器和电子表格，通常也可用于采集数据。在下一节，我们将着眼于如何从这些来源提取数 据。

另一种语料库创建方案是典型的实验研究，其中一些精心设计的材料被从一定范围的人 类受试者中收集，然后进行分析来评估一个假设或开发一种技术。此类数据库在实验室或公 司内被共享和重用已很常见，经常被更广泛的发布。这种类型的语料库是“共同任务”的科研 管理方法的基础，这在过去的二十年已成为政府资助的语言技术研究项目。在前面的章节中， 我们已经遇到很多这样的语料库；我们将看到如何编写Python 程序实践这些语料库发布前 必要的一些任务。 

最后，还有努力为一个特定的语言收集“参考语料”，如美国国家语料库（American National Corpus，ANC）和英国国家语料库（British National Corpus，BNC）。这里的目标已 经成为产生各种形式、风格和语言的使用的一个全面的记录。除了规模庞大的挑战，还严重 依赖自动标注工具和后期编辑共同修复错误。然而，我们可以编写程序来查找和修复错误， 还可以分析语料库是否平衡。

#### 质量控制

自动和手动的数据准备的好的工具是必不可少的。然而，一个高质量的语料库的建立很 大程度取决于文档、培训和工作流程等平凡的东西。标注指南确定任务并记录标记约定。它 们可能会定期更新以覆盖不同的情况，同时制定实现更一致的标注的新规则。在此过程中标 注者需要接受训练，包括指南中没有的情况的解决方法。需要建立工作流程，尽可能与支持 软件一起，跟踪哪些文件已被初始化、标注、验证、手动检查等等。可能有多层标注，由不 同的专家提供。不确定或不一致的情况可能需要裁决。


#### 维护与演变

### 11.3 数据采集

网络是语言分析的一个丰富的数据源。我们已经讨论了访问单个文件，如RSS 订阅、 搜索引擎的结果（见3.1节）的方法。然而，在某些情况下，我们要获得大量的Web 文本。

最简单的方法是获得出版的网页文本的文集。

#### 从字处理器文件获取数据

文字处理软件通常用来在具有有限的可计算基础设施的项目中手工编制文本和词汇。这 些项目往往提供数据录入模板，通过字处理软件并不能保证数据结构正确。例如：每个文本 可能需要有一个标题和日期。同样，每个词条可能有一些必须的字段。随着数据规模和复杂 性的增长，用于维持其一致性的时间的比重也增大。 

我们怎样才能提取这些文件的内容，使我们能够在外部程序中操作？此外，我们如何才 能验证这些文件的内容，以帮助作者创造结构良好的数据，在原始的创作过程中最大限度提 高数据的质量？

考虑一个字典，其中的每个条目都有一个词性字段，从一个 20 个可能值的集合选取， 在发音字段显示，以 11号黑体字呈现。传统的文字处理器没有能够验证所有的词性字段已 正确输入和显示的搜索函数或宏。这个任务需要彻底的手动检查。如果字处理器允许保存文 档为一种非专有的格式，如 text、HTML 或XML，有时我们可以写程序自动做这个检查。 

思考下面的一个词条的片段：“sleep [sli:p] v.i. condition of body and mind...”。我们可以在MSWord 中输入这些词，然后“另存为网页”，然后检查生成的 HTML 文件：
```
<p class=MsoNormal>sleep 
<span style='mso-spacerun:yes'> </span>
[<span class=SpellE>sli:p</span>]
<span style='mso-spacerun:yes'> </span> 
<b><span style='font-size:11.0pt'>v.i.</span></b> 
<span style='mso-spacerun:yes'> </span> 
<i>a condition of body and mind ...<o:p></o:p></i> 
</p>
```
观察该条目的 HTML 段落表示，使用了`<p>`元素，词性出现在`<span style='font-size:11.0pt'>`元素内。下面的程序定义了合法的词性的集合：`legal_pos`。然后从 `dict.htm`文件提取所有11号字的内容，并存储在集合 `used_pos`中。请看搜索模式包含一个括号括起来的子表达式；只有匹配该子表达式的材料才会被 `re.findall` 返回。最后，程序用 `used_pos` 和 `legal_pos` 的补集构建非法词性的集合：

In [6]:
import re
legal_pos = set(['n', 'v.t.', 'v.i.', 'adj', 'det'])
pattern = re.compile(r"'font-size:11.0pt'>([a-z.]+)<")
document = open("dict.htm").read()
used_pos = set(re.findall(pattern, document))
illegal_pos = used_pos.difference(legal_pos)
print(list(illegal_pos))

[]


In [11]:
# 例11-1. 将Microsoft Word创建的HTML转换成CSV
import re
from bs4 import BeautifulSoup
def lexical_data(html_file):
    SEP = '_ENTRY'
    html = open(html_file).read()
    html = re.sub(r'<p', SEP + '<p', html)
    # text = nltk.clean_html(html)
    text = BeautifulSoup(html).get_text()
    text = ' '.join(text.split())
    for entry in text.split(SEP):
        if entry.count(' ') > 2:
            yield entry.split(' ', 3)
import csv
writer = csv.writer(open("dict1.csv", "w"))
writer.writerows(lexical_data('dict.htm'))

#### 从电子表格和数据库中获取数据

电子表格通常用于获取**词表或范式**。例如：一个比较词表可以用电子表格创建，用一排表示每个同源组，每种语言一列（见 `nltk.corpus.swadesh`和 `www.rosettaproject.org`）。大多数电子表格软件可以将数据导出为 `CSV` 格式。正如我们将在下面看到的，使用 `csv` 模块 Python 程序可以很容易的访问它们。

有时词典存储在一个完全成熟的关系数据库。经过适当的标准化，这些数据库可以确保数据的有效性。例如：我们可以要求所有词性都来自指定的词汇，通过声明词性字段为枚举类型或用一个外键引用一个单独的词性表。然而，关系模型需要提前定义好的数据（模式）结构，这与高度探索性的构造语言数据的主导方法相违背。被认为是强制性的和独特的字段往往需要是可选的、可重复。只有当数据类型提前全都知道时关系数据库才是适用的。如果不是，或者几乎所有的属性都是可选的或重复的，关系的做法就行不通了。

然而，当我们的目标只是简单的从数据库中提取内容时，完全可以将表格（或 SQL 查询结果）转换成 CSV 格式，并加载到我们的程序中。我们的程序可能会执行不太容易用 SQL 表示的语言学目的的查询，如：`select all words that appear in example sentences for which no dictionary entry is provided`。对于这个任务，我们需要从记录中提取足够的信息，使它连同词条和例句能被唯一的识别。让我们假设现在这个信息是在一个 `CSV` 文件 `dict.csv` 中：
```
"sleep","sli:p","v.i","a condition of body and mind ..."
"walk","wo:k","v.intr","progress by lifting and setting down each foot ..." "wake","weik","intrans","cease to sleep"
```
现在，我们可以表示此查询，如下所示：

In [13]:
import csv
lexicon = csv.reader(open('dict.csv'))
pairs = [(lexeme, defn) for (lexeme, _, _, defn) in lexicon]
lexemes, defns = zip(*pairs)
defn_words = set(w for defn in defns for w in defn.split())
sorted(defn_words.difference(lexemes))

['...',
 'a',
 'and',
 'body',
 'by',
 'cease',
 'condition',
 'down',
 'each',
 'foot',
 'lifting',
 'mind',
 'of',
 'progress',
 'setting',
 'to']

#### 转换数据格式

已标注语言数据很少以最方便的格式保存，往往需要进行各种格式转换。字符编码之间的转换已经讨论过（见 3.3节）。在这里，我们专注于数据结构。 最简单的情况，输入和输出格式是同构的。例如：我们可能要将词汇数据从 `Toolbox` 格式转换为 `XML` ，可以直接一次一个的转换词条（11.4 节）。数据结构反映在所需的程序的结构中：一个 `for` 循环，每次循环处理一个词条。 另一种常见的情况，输出是输入的摘要形式，如：**一个倒置的文件索引。有必要在内存中建立索引结构**（见例 4.8），然后把它以所需的格式写入一个文件。下面的例子构造一个索引，映射字典定义的词汇到相应的每个词条的语意，已经对定义文本分词，并丢弃短词。一旦该索引建成，我们打开一个文件，然后遍历索引项，以所需的格式输出行。

In [14]:
idx = nltk.Index((defn_word, lexeme)
                for (lexeme, defn) in pairs
                for defn_word in nltk.word_tokenize(defn)
                if len(defn_word) > 3)
idx_file = open("dict.idx", "w")
for word in sorted(idx):
    idx_words = ', '.join(idx[word])
    idx_line = "%s: %s\n" % (word, idx_words)
    idx_file.write(idx_line)
idx_file.close()

#### 决定要包含的标注层

发布的语料库中所包含的信息的丰富性差别很大。语料库最低限度通常会包含至少一个声音或字形符号的序列。事情的另一面，一个语料库可以包含大量的信息，如：句法结构、形态、韵律、每个句子的语义、加上段落关系或对话行为的标注。标注的这些额外的层可能正是有人执行一个特定的数据分析任务所需要的。例如：如果我们可以搜索特定的句法结构，找到一个给定的语言模式就更容易；如果每个词都标注了意义，为语言模式归类就更容易。 

这里提供一些常用的标注层： 
- 分词：文本的书写形式不能明确地识别它的标识符。分词和规范化的版本作为常规的正式版本的补充可能是一个非常方便的资源。 
- 断句：正如我们在第 3章中看到的，断句比它看上去的似乎更加困难。因此，一些语料库使用明确的标注来断句。 
- 分段：段和其他结构元素（标题，章节等）可能会明确注明。 
- 词性：文档中的每个单词的词类。 
- 句法结构：一个树状结构，显示一个句子的组成结构。
- 浅层语义：命名实体和共指标注，语义角色标签。 
- 对话与段落：对话行为标记，修辞结构。

不幸的是，现有的语料库之间在如何表示标注上并没有多少一致性。然而，两个大类的标注表示应加以区别。**内联标注**通过插入带有标注信息的特殊符号或控制序列修改原始文档。例如：为文档标注词性时，字符串“fly”可能被替换为字符串“fly/NN”来表示词 fly 在文中是名词。相比之下，**对峙标注**不修改原始文档，而是创建一个新的文档，通过使用指针引用原始文档来增加标注信息。例如：这个新的文档可能包含字符串`“<token id=8 pos=' NN'/>”`，表示 8 号标识符是一个名词。

### 11.4 使用 XML

可扩展标记语言（`The Extensible Markup Language，XML`）为设计特定领域的标记语言提供了一个框架。它有时被用于表示已标注的文本和词汇资源。不同于 HTML 的标签是预定义的，XML 允许我们组建自己的标签。不同于数据库，XML 允许我们创建的数据而不必事先指定其结构，它允许我们有可选的、可重复的元素。在本节中，我们简要回顾一下 XML 的一些与表示语言数据有关的特征，并说明如何使用Python 程序访问 XML 文件中存储的数据。

### `ElementTree` 接口

Python的 `ElementTree` 模块提供了一种方便的方式访问存储在 XML 文件中的数据。

我们将使用 XML 格式的莎士比亚戏剧集来说明 ElementTree 的使用方法。让我们加载 XML 文件并检查原始数据，首先在文件的顶部，在那里我们看到一些XML 头和一个名为 `play.dtd` 的架构，接着是根元素 `PLAY`。我们从 `Act 1`再次获得数据。

In [17]:
merchant_file = nltk.data.find('corpora/shakespeare/merchant.xml')
raw = open(merchant_file).read()
print(raw[0:168])

<?xml version="1.0"?>
<?xml-stylesheet type="text/css" href="shakes.css"?>
<!-- <!DOCTYPE PLAY SYSTEM "play.dtd"> -->

<PLAY>
<TITLE>The Merchant of Venice</TITLE>

<!-


下一步是作为结构化的 XML 数据使用 `ElementTree` 处理文件的内容。我们正在处理一个文件（一个多行字符串），并建立一棵树，所以方法的名称是 `parse` 并不奇怪。变量 `merchant` 包含一个 XML 元素 `PLAY`。此元素有内部结构；我们可以使用一个索引来得到它的第一个孩子，一个 `TITLE` 元素。我们还可以看到该元素的文本内容：戏剧的标题。 要得到所有的子元素的列表，我们使用 `getchildren()`方法。

In [20]:
from nltk.corpus import toolbox
lexicon = toolbox.xml('rotokas.dic')
lexicon[3][0]

<Element 'lx' at 0x0000029BC92F89F8>

In [21]:
lexicon[3][0].tag

'lx'

In [22]:
lexicon[3][0].text

'kaa'

### 11.5 使用 Toolbox 数据


In [23]:
# 我们可以为每个条目计算字段的平均个数：
from nltk.corpus import toolbox
lexicon = toolbox.xml('rotokas.dic')
sum(len(entry) for entry in lexicon) / len(lexicon)

13.635955056179775

#### 为每个条目添加一个字段

In [25]:
# 例11-2. 为词汇条目添加新的cv字段
from xml.etree.ElementTree import SubElement
def cv(s):
    s = s.lower()
    s = re.sub(r'[^a-z]', r'_', s)
    s = re.sub(r'[aeiou]', r'V', s)
    s = re.sub(r'[^V_]', r'C', s)
    return (s)

def add_cv_filed(entry):
    for filed in entry:
        if filed.tag == 'lx':
            cv_field = SubElement(entry, 'cv')
            cv_field.text = cv(filed.text)
lexicon = toolbox.xml("rotokas.dic")
add_cv_filed(lexicon[53])

### 11.6 使用 `OLAC` 元数据描述语言资源

元数据最简单的定义是“关于数据的结构化数据”。元数据是对象或资源的描述信息，无论是物理的还是电子的。

#### OLAC：开放语言档案社区

开放语言档案社区（·Open Language Archives Community，OLAC·）是正在创建的一个世界性语言资源的虚拟图书馆的机构和个人的国际伙伴关系：（i）制订目前最好的关于语言 资源的数字归档实施的共识，（ii）开发存储和访问这些资源的互操作信息库和服务的网络。 OLAC 的主页在 `http://www。language-archives.org/`。

### 11.7 小结

- 大多数语料库中基本数据类型是已标注的文本和词汇。文本有时间结构，而词汇有记录结构。
- 语料库的生命周期，包括数据收集、标注、质量控制以及发布。发布后生命周期仍然继续，因为语料库会在研究过程中被修改和丰富。
- 语料库开发包括捕捉语言使用的代表性的样本与使用任何一个来源或文体都有足够的材料之间的平衡；增加变量的维度通常由于资源的限制而不可行。
- `XML` 提供了一种有用的语言数据的存储和交换格式，但解决普遍存在的数据建模问题没有捷径。
- `Toolbox` 格式被广泛使用在语言记录项目中；我们可以编写程序来支持 `Toolbox` 文件的 维护，将它们转换成`XML`。
- 开放语言档案社区（`OLAC`）提供了一个用于记录和发现语言资源的基础设施。